# HDP Gibbs Samplers

## Implementation of Samplers

Both of the above schemes are implemented in the `HDP` class and callable via the `gibbs_crf` and `gibbs_direct` methods, respectively.

In [5]:
import numpy as np
import pandas as pd
from scipy import sparse
from scipy.special import gammaln as logg

In [6]:
def pois_fk_cust(i, x, k, Kmax, ha, hb, new=False):
    """
    Computes the mixture components for a given customer across all k values.
    MODEL: base measure H ~ Gamma(ha, hb), F(x|phi) ~ Poisson(phi)
    All components are calculated exactly in log-space and then exponentiated.
    
    returns: (Kmax,) vector; if new=True, returns a scalar
    """
    
    x = x.flatten()  # reshape to 1D, since gibbs routine passes in a 2D array
    
    # Calculate the case where k has no members
    fknew_cust = np.exp( -logg(x[i] + 1) + logg(x[i] + ha) - logg(ha) -
                         (x[i] + ha)*np.log(1 + hb) + ha*np.log(hb) )
    if new == True: return fknew_cust        
    
    x_kks = [x[k == kk] for kk in range(Kmax)]  # subset of customers eating kk
    xi_in = np.zeros(Kmax)                      # offset if x[i] is in this subset
    xi_in[k[i]] = 1
      
    # Compute (a,b) params from gamma kernel tricks done in fk function
    av = np.array(list(map(np.sum, x_kks))) - xi_in*x[i] + ha
    bv = np.array(list(map(len, x_kks))) - xi_in + hb
    fk_cust = np.exp( -logg(x[i] + 1) + logg(x[i] + av) - logg(av) -
                      (x[i] + av)*np.log(1 + bv) + av*np.log(bv) )
     
    return fk_cust


def pois_fk_tabl(jj, tt, x, j, t, k, Kmax, ha, hb, new=False):
    """
    Computes the mixture components for a given table across all k values.
    MODEL: base measure H ~ Gamma(ha, hb), F(x|phi) ~ Poisson(phi)
    All components are calculated exactly in log-space and then exponentiated.
    
    returns: (Kmax,) vector; if new=True, returns a scalar
    """
    
    x = x.flatten()  # reshape to 1D, since gibbs routine passes in a 2D array
    x_jt = x[np.logical_and(j == jj, t == tt)]
    kk = k[np.logical_and(j == jj, t == tt)]
    
    fknew_tabl = np.exp( -np.sum(logg(x_jt + 1)) + logg(np.sum(x_jt) + ha) - logg(ha) -
                         (np.sum(x_jt) + ha)*np.log(len(x_jt) + hb) + ha*np.log(hb) )
    # If table jt doesn't exist, just return the "new" mixture component
    if len(x_jt) == 0:
        #print(f"WARNING: table {(jj, tt)} does not exist currently")
        new = True
    if new == True: return np.full(Kmax, fknew_tabl)
    
    x_kks = [x[k == kk] for kk in range(Kmax)]  # subset of customers at tables serving kk
    xjt_in = np.zeros(Kmax)                     # offset if table x_jt is in this subset
    xjt_in[kk[0]] = 1
      
    # Compute (a,b) params from gamma kernel tricks done in fk function
    av = np.array(list(map(np.sum, x_kks))) - xjt_in*np.sum(x_jt) + ha
    bv = np.array(list(map(len, x_kks))) - xjt_in*len(x_jt) + hb
    fk_tabl = np.exp( -np.sum(logg(x_jt + 1)) + logg(np.sum(x_jt) + av) - logg(av) -
                       (np.sum(x_jt) + av)*np.log(len(x_jt) + bv) + av*np.log(bv) )
     
    return fk_tabl

In [7]:
def mnom_fk_cust(i, x, k, Kmax, L, ha, new=False):
    """
    Computes the mixture components for a given customer across all k values.
    MODEL: base measure H ~ Dirichlet(L, ha_1,...,ha_L),
                        F(x|phi) ~ Multinomial(n_ji, phi_1,...,phi_L)
    All components are calculated exactly in log-space and then exponentiated.
    X can be a dense or a sparse csr-style matrix.
    
    returns: (Kmax,) vector; if new=True, returns a scalar
    """
    
    xi, ni = x[i, :], np.sum(x[i, :])
    log_con = logg(ni + 1) - np.sum(logg(xi + np.ones(L))) # term constant for all k
    # Calculate the case where k has no members
    
    if new == True:
        fknew_cust = np.exp( log_con + np.sum(logg(xi + ha)) - logg(np.sum(xi + ha)) + 
                             logg(np.sum(ha)) - np.sum(logg(ha)) )
        return fknew_cust        
    
    # Get subset of customers eating kk; each entry is a (#, L) matrix
    x_kks = [x[k == kk, :] for kk in range(Kmax)]  
    
    # Compute params from Dirichlet kernel tricks done in fk function
    a_bot = np.vstack([np.sum(x_kk, axis=0) for x_kk in x_kks]) + ha[None, :]    # (Kmax, L)
    a_bot[k[i], :] -= xi                         # offset if xi is in this subset
    a_top = np.apply_along_axis(lambda row: row + xi, 1, a_bot)
    fk_cust = np.exp( log_con + np.sum(logg(a_top), axis=1) - logg(np.sum(a_top, axis=1)) +
                      logg(np.sum(a_bot, axis=1)) - np.sum(logg(a_bot), axis=1) )
     
    # Convert back to a dense array in case X was sparse
    return np.asarray(fk_cust).ravel()


def mnom_fk_tabl(jj, tt, x, j, t, k, Kmax, L, ha, new=False):
    """
    Computes the mixture components for a given customer across all k values.
    MODEL: base measure H ~ Dirichlet(L, ha_1,...,ha_L),
                        F(x|phi) ~ Multinomial(n_ji, phi_1,...,phi_L)
    All components are calculated exactly in log-space and then exponentiated.
    
    returns: (Kmax,) vector; if new=True, returns a scalar
    """
    
    x_jt = x[np.logical_and(j == jj, t == tt), :]                                # (|T|, L)
    kk = k[np.logical_and(j == jj, t == tt)]
    n_jt = np.sum(x_jt, axis=1)                                                  # (|T|,)
    sum_jt = np.sum(x_jt, axis=0)                                                # (L,)
    log_con = np.sum(logg(n_jt + 1)) - np.sum(logg(x_jt + 1))    # term constant for all k
    
    fknew_tabl = np.exp( log_con + np.sum(logg(sum_jt + ha)) - logg(np.sum(sum_jt + ha)) + 
                         logg(np.sum(ha)) - np.sum(logg(ha)) )
    # If table jt doesn't exist, just return the "new" mixture component
    if x_jt.shape[0] == 0:
        #print(f"WARNING: table {(jj, tt)} does not exist currently")
        new = True
    if new == True: return fknew_tabl       
    
    # Get subset of customers eating kk; each entry is a (#, L) matrix
    x_kks = [x[k == kk, :] for kk in range(Kmax)]
      
    # Compute params from Dirichlet kernel tricks done in fk function
    a_bot = np.vstack([np.sum(x_kk, axis=0) for x_kk in x_kks]) + ha[None, :]    # (Kmax, L)
    a_bot[kk[0], :] -= sum_jt                       # offset if table x_jt is in this subset
    a_top = a_bot + sum_jt[None, :]
    fk_tabl = np.exp( log_con + np.sum(logg(a_top), axis=1) - logg(np.sum(a_top, axis=1)) +
                      logg(np.sum(a_bot, axis=1)) - np.sum(logg(a_bot), axis=1) )

    return fk_tabl

In [8]:
def cat_fk_cust(i, x, k, Kmax, L, ha, new=False):
    """
    Computes the mixture components for a given customer across all k values.
    MODEL: base measure H ~ Dirichlet(L, ha_1,...,ha_L),
                        F(x|phi) ~ Categorical(L, phi_1,...,phi_L)
    All components are calculated exactly in log-space and then exponentiated.
    X can be a dense or a sparse csr-style matrix.
    
    returns: (Kmax,) vector; if new=True, returns a scalar
    """
    
    xi = x[i, :]
    ll = sparse.find(xi)[0][0]        # get index of the 1 value
    # Calculate the case where k has no members
    if new == True:
        return ha[ll] / np.sum(ha)    
    
    # Store the size of sets V and V_l for each k
    V_kks = np.array([np.sum(k == kk) for kk in range(Kmax)])
    Vl_kks = np.array([np.sum(x[k == kk, ll]) for kk in range(Kmax)])
    
    fk_cust = (Vl_kks + ha[ll]) / (V_kks + np.sum(ha))
    return fk_cust


def cat_fk_tabl(jj, tt, x, j, t, k, Kmax, L, ha, new=False):
    """
    Computes the mixture components for a given customer across all k values.
    MODEL: base measure H ~ Dirichlet(L, ha_1,...,ha_L),
                        F(x|phi) ~ Categorical(L, phi_1,...,phi_L)
    All components are calculated exactly in log-space and then exponentiated.
    
    returns: (Kmax,) vector; if new=True, returns a scalar
    """
    
    x_jt = x[np.logical_and(j == jj, t == tt), :]                                # (|T|, L)
    kk = k[np.logical_and(j == jj, t == tt)]
    n_jt = np.sum(x_jt, axis=1)                                                  # (|T|,)
    sum_jt = np.sum(x_jt, axis=0)                                                # (L,)
    log_con = np.sum(logg(n_jt + 1)) - np.sum(logg(x_jt + 1))    # term constant for all k
    
    fknew_tabl = np.exp( log_con + np.sum(logg(sum_jt + ha)) - logg(np.sum(sum_jt + ha)) + 
                         logg(np.sum(ha)) - np.sum(logg(ha)) )
    # If table jt doesn't exist, just return the "new" mixture component
    if x_jt.shape[0] == 0:
        print(f"WARNING: table {(jj, tt)} does not exist currently")
        new = True
    if new == True: return fknew_tabl       
    
    # Get subset of customers eating kk; each entry is a (#, L) matrix
    x_kks = [x[k == kk, :] for kk in range(Kmax)]
      
    # Compute params from Dirichlet kernel tricks done in fk function
    a_bot = np.vstack([np.sum(x_kk, axis=0) for x_kk in x_kks]) + ha[None, :]    # (Kmax, L)
    a_bot[kk[0], :] -= sum_jt                       # offset if table x_jt is in this subset
    a_top = a_bot + sum_jt[None, :]
    fk_tabl = np.exp( log_con + np.sum(logg(a_top), axis=1) - logg(np.sum(a_top, axis=1)) +
                      logg(np.sum(a_bot, axis=1)) - np.sum(logg(a_bot), axis=1) )

    return fk_tabl

In [9]:
class StirlingEngine:
    """
    Numerically efficient engine for computing and storing computed Stirling numbers.
    
    CONSTRUCTOR PARAMETERS
    - Nmax: largest integer n for which s(n,m) will need to be computed
    
    PRIVATE ATTRIBUTES
    - s_memo_, slog_memo_: running tables of previously computed values
    """
    
    def __init__(self, Nmax):
        self.s_memo_ = np.full((Nmax, Nmax), np.nan)
        self.slog_memo_ = np.full((Nmax, Nmax), np.nan)
        
        
    def stirling(self, n, m):
        """
        Computes an unsigned Stirling number of the first kind.
        Uses dynamic programming to store previously computed s(n,m) values,
        as this is a repeatedly-called recursive algorithm.
        """
        
        assert n < self.s_memo_.shape[0] and m < self.s_memo_.shape[0]  
        # If this has already been computed, return stored value
        if not np.isnan(self.s_memo_[n, m]):
            return self.s_memo_[n, m]
        else:
            return_val = np.nan

            # Base cases
            if (n == 0 and m == 0) or (n == 1 and m == 1):
                return_val = 1
            elif (n > 0 and m == 0) or m > n:
                return_val = 0
            # Recursion relation
            else:
                return_val = self.stirling(n-1, m-1) + (n-1)*self.stirling(n-1, m)

            self.s_memo_[n, m] = return_val
            return return_val
    
    
    def stirlog(self, n, m):
        """
        Computes the natural logarithm of an unsigned Stirling number,
        using the same dynamic programming approach as above.
        If s(n,m) = 0, this gets returned as -inf (np.exp(-inf) == 0.0)
        
        This is the preferred function, as stirling() can encounter overflow errors.
        """
        
        assert n < self.slog_memo_.shape[0] and m < self.slog_memo_.shape[0]  
        # If this has already been computed, return stored value
        if not np.isnan(self.slog_memo_[n, m]):
            return self.slog_memo_[n, m]
        else:
            return_val = np.nan

            # Base cases
            if (n == 0 and m == 0) or (n == 1 and m == 1):
                return_val = 0
            elif (n > 0 and m == 0) or m > n:
                return_val = -np.inf
            # Recursion relation
            else:
                log_s1, log_s2 = self.stirlog(n-1, m-1), self.stirlog(n-1, m)
                # If s1 == 0 (log_s1 == -inf), just return (n-1)*log_s2
                # By definition, must have s2 > s1, so only need to check s1
                if np.isfinite(log_s1):
                    val = (n-1) * np.exp(log_s2 - log_s1)
                    # If there is overflow/underflow in `val`, approximate log(1+x) = log(x)
                    if np.isfinite(val):
                        return_val = log_s1 + np.log1p(val)
                    else:
                        return_val = log_s2 + np.log(n-1)
                else:
                    return_val = log_s2 + np.log(n-1)

            self.slog_memo_[n, m] = return_val
            return return_val

In [73]:
class HDP:
    """
    Model implementing the Chinese Restaurant Franchise Process formulation of the HDP.
    
    CONSTRUCTOR PARAMETERS
    - gamma, alpha0: scaling parameters > 0 for base measures H and G0
    - f: string representing distribution of data; h is chosen to be conjugate
    - hypers: tuple of hyperparameter values specific to f/h scheme chosen
    
    PRIVATE ATTRIBUTES (volatile)
    - tk_map_: (J x Tmax) matrix of k values for each (j,t) pair
    - beta_: (Kmax + 1,) vector of beta values for each k
    - n_: (J x Tmax) matrix specifying counts of customers (gibbs_cfr)
    - q_: (J x Kmax) matrix specifying counts of customers (gibbs_direct)
    - m_: (J x Kmax) matrix specifying counts of tables
    - fk_cust_, fk_tabl_: functions to compute mixing components for Gibbs sampling
    - stir_: an object of class StirlingEngine which computes Stirling numbers
    
    PUBLIC ATTRIBUTES
    cfr_samples: (S x N x 2) matrix of (t, k) values for each data point i;
                 exists only after gibbs_cfr() has been called
    direct_samples: (S x N) matrix of k values for each data point i;
                    exists only after gibbs_direct() has been called
    beta_samples: (S x Kmax+1) matrix of beta values after each iteration;
                  exists only after gibbs_direct() has been called
    """
    
    def __init__(self, gamma=1, alpha0=1, f='multinomial', hypers=None):
        self.g_ = gamma
        self.a0_ = alpha0
        self.set_priors(f, hypers)
        
    def set_priors(self, f, hypers):
        """
        Initializes the type of base measure h_ and data-generation function f_.
        Also sets hypers_, the relevelant hyperparameters and
                  fk_routine_, the function to compute mixing components.
        """
        if f == 'poisson':
            # Specify parameters of H ~ Gamma(a,b)
            if hypers is None:
                self.hypers_ = (1,1)
            else: self.hypers_ = hypers
            self.fk_cust_ = pois_fk_cust
            self.fk_tabl_ = pois_fk_tabl
        
        elif f == 'multinomial':
            if hypers is None:
                L = 2
                self.hypers_ = (L, np.ones(L))
            else: self.hypers_ = hypers
            self.fk_cust_ = mnom_fk_cust
            self.fk_tabl_ = mnom_fk_tabl
            
        elif f == 'categorical':
            # Identical to multinomial, but with some efficiency upgrades
            if hypers is None:
                L = 2
                self.hypers_ = (L, np.ones(L))
            else: self.hypers_ = hypers
            self.fk_cust_ = cat_fk_cust
            self.fk_tabl_ = mnom_fk_tabl
    
    
    def tally_up(self, it, which=None):
        """
        Helper function for computing maps and counts in gibbs().
        Given a current iteration in the cfr_samples attribute, does a full
        recount of customer/table allocations, updating n_ and m_.
        Set which = 'n' or 'm' to only tally up that portion
        """    
        
        if which == 'n':
            jt_pairs = self.cfr_samples[it,:,0:2]
            # Count customers at each table (jt)
            cust_counts = pd.Series(map(tuple, jt_pairs)).value_counts()
            j_idx, t_idx = tuple(map(np.array, zip(*cust_counts.index)))
            self.n_ *= 0
            self.n_[j_idx, t_idx] = cust_counts
            
        elif which == 'm':
            jt_pairs = self.cfr_samples[it,:,0:2]
            # First filter by unique tables (jt), then count tables with each k value
            jt_unique, k_idx = np.unique(jt_pairs, axis=0, return_index=True)
            jk_pairs = np.c_[self.cfr_samples[it, k_idx, 0],
                             self.cfr_samples[it, k_idx, 2]]
            #print(jk_pairs)
            tabl_counts = pd.Series(map(tuple, jk_pairs)).value_counts()
            #print(tabl_counts)
            j_idx, k_idx = tuple(map(np.array, zip(*tabl_counts.index)))
            self.m_ *= 0
            self.m_[j_idx, k_idx] = tabl_counts
            
        elif which == 'q':
            jk_pairs = self.direct_samples[it,:,:]
            # Counts customers at each j eating k
            cust_counts = pd.Series(map(tuple, jk_pairs)).value_counts()
            j_idx, k_idx = tuple(map(np.array, zip(*cust_counts.index)))
            self.q_ *= 0
            self.q_[j_idx, k_idx] = cust_counts
            
    
    def get_dist(self, old, new, used, size):
        """
        Helper function which standardizes the operation of computing a
        full conditional distribution, for both t and k values.
        Also normalizes and ensures there are no NANs.
        - old: a (size,) vector of probability values for used values
        - new: a scalar representing the combined probability of all unused values
        - used: a (size,) mask encoding which values in the sample space are being used
        - size: the size of the sample space
        """
        
        num_unused = size - np.sum(used)
        dist = None
        if num_unused == 0:
            # In our truncated sample space, there is no room for "new" values
            dist = old
        else:
            dist = old * used + (new / num_unused) * np.logical_not(used)
        
        # Remove nans and add epsilon so that distribution is all positive
        dist[np.isnan(dist)] = 0
        dist += 1e-10
        return dist / np.sum(dist)
    
    
    def draw_t(self, it, x, j, Tmax, Kmax, verbose):
        """
        Helper function which does the draws from the t_ij full conditional.
        Updates the counts and the samples matrices at iteration `it`.
        Called by gibbs_cfr()
        """
        
        t_next, k_next = self.cfr_samples[it,:,1], self.cfr_samples[it,:,2]
        # Cycle through the t value of each customer, conditioning on everything
        # Randomize the order in which updates occur
        for i in np.random.permutation(len(j)):
            jj, tt0, kk0 = j[i], t_next[i], k_next[i]

            # Get vector of customer f_k values (dependent on model specification)
            old_mixes = self.fk_cust_(i, x, k_next, Kmax, *self.hypers_) 
            new_mixes = self.fk_cust_(i, x, k_next, Kmax, *self.hypers_, new=True) 
            # Calculate pointwise likelihoods p(x_ji | ...)
            M = np.sum(self.m_)
            Mk = np.sum(self.m_, axis=0)   # number of tables serving k
            lik = old_mixes @ (Mk / (M + self.g_)) + new_mixes * (self.g_ / (M + self.g_))

            cust_offset = np.zeros(Tmax)
            cust_offset[tt0] = 1
            old_t = (self.n_[jj, :] - cust_offset) * old_mixes[self.tk_map_[jj, :]]      
            new_t = self.a0_ * lik
            # If a table is in use, prob comes from old_t; otherwise, from new_t
            # Distribute the weight of new_t across all possible new allocations
            t_used = self.n_[jj, :] > 0
            t_dist = self.get_dist(old_t, new_t, t_used, Tmax)

            tt1 = np.random.choice(Tmax, p=t_dist)
            t_next[i] = tt1
            self.tally_up(it, which='n')

            # If this table was previously unoccupied, we need to select a k
            if self.n_[jj, tt1] == 1 and tt0 != tt1:
                old_k = np.sum(self.m_, axis=0) * old_mixes
                new_k = self.g_ * new_mixes
                k_used = np.sum(self.m_, axis=0) > 0
                k_dist = self.get_dist(old_k, new_k, k_used, Kmax)

                kk1 = np.random.choice(Kmax, p=k_dist)
                self.tk_map_[jj, tt1] = kk1
                k_next[i] = self.tk_map_[jj, tt1]
            self.tally_up(it, which='m')

            #if verbose: print(f"~ customer (j,i) = {(jj,i)}" +
                              #f" moves table: {tt0} -> {t_next[i]}, k: {kk0} -> {k_next[i]}")
    
    
    def draw_k(self, it, x, j, Kmax, verbose):
        """
        Helper function which does the draws from the t_ij full conditional.
        Updates the counts and the samples matrices at iteration `it`.
        Called by gibbs_cfr()
        """
        
        t_next, k_next = self.cfr_samples[it,:,1], self.cfr_samples[it,:,2]
        # Cycle through the k values of each table
        j_idx, t_idx = np.where(self.n_ > 0)   # find the occupied tables
        for i in np.random.permutation(len(j_idx)):
            jj, tt = j_idx[i], t_idx[i]
            kk0 = self.tk_map_[jj, tt]

            # Get vector of table f_k values (dependent on model specification)
            old_mixes = self.fk_tabl_(jj, tt, x, j, t_next, k_next, Kmax, *self.hypers_) 
            new_mixes = self.fk_tabl_(jj, tt, x, j, t_next, k_next, Kmax, *self.hypers_, new=True) 

            tabl_offset = np.zeros(Kmax)
            tabl_offset[kk0] = 1
            old_k = (np.sum(self.m_, axis=0) - tabl_offset) * old_mixes
            new_k = self.g_ * new_mixes
            k_used = np.sum(self.m_, axis=0) > 0
            k_dist = self.get_dist(old_k, new_k, k_used, Kmax)

            kk1 = np.random.choice(Kmax, p=k_dist)
            self.tk_map_[jj, tt] = kk1
            k_next[np.logical_and(j == jj, t_next == tt)] = kk1
            self.tally_up(it, which='m')

            #if verbose: print(f"~~ table (j,t) = {(jj,tt)} changes dish: {kk0} -> {kk1}")
    
    
    def draw_z(self, it, x, j, Kmax, verbose):
        """
        Helper function which does the draws from the z_ij full conditional.
        Updates the counts and the samples matrices at iteration `it`.
        Called by gibbs_direct()
        """
        
        k_next = self.direct_samples[it,:,1]
        # Cycle through the k values of each customer
        for i in np.random.permutation(len(j)):
            jj, kk0 = j[i], k_next[i]
            
            # Get vector of customer f_k values (dependent on model specification)
            old_mixes = self.fk_cust_(i, x, k_next, Kmax, *self.hypers_) 
            new_mixes = self.fk_cust_(i, x, k_next, Kmax, *self.hypers_, new=True) 
            
            cust_offset = np.zeros(Kmax)
            cust_offset[kk0] = 1
            old_k = (self.q_[jj, :] - cust_offset +
                     self.a0_ * self.beta_samples[it, :-1]) * old_mixes      
            new_k = self.a0_ * self.beta_samples[it, -1] * new_mixes
            k_used = np.sum(self.m_, axis=0) > 0
            k_dist = self.get_dist(old_k, new_k, k_used, Kmax)

            kk1 = np.random.choice(Kmax, p=k_dist)
            k_next[i] = kk1
            self.tally_up(it, which='q')
            
            # If this k value was previously unused, must also set the beta_k component
            if np.sum(self.q_[:, kk1] == 1):
                b = np.random.beta(1, self.g_)
                beta_u = self.beta_samples[it, -1]
                self.beta_samples[it, kk1] = b * beta_u
                self.beta_samples[it, -1] = (1-b) * beta_u
            
            #if verbose:
                #print(f"~ customer (j,i) = {(jj,i)}" +
                      #f"changes dish: {kk0} -> {kk1}")
                #print(f"  k_dist: {k_dist.round(3)} (sum {np.sum(k_dist)})")
                
    
    def draw_m(self, it, x, j, Kmax, verbose):
        """
        Helper function which does the draws from the z_ij full conditional.
        Updates the counts and the samples matrices at iteration `it`.
        Called by gibbs_direct()
        """
        
        k_next = self.direct_samples[it,:,1]
        self.m_ *= 0                           # reset the m counts
        # Cycle through the k values of each restaurant
        j_idx, k_idx = np.where(self.q_ > 0)   # find the consumed dishes
        for i in np.random.permutation(len(j_idx)):
            jj, kk = j_idx[i], k_idx[i]
            max_m = self.q_[jj, kk]
            
            abk = self.a0_ * self.beta_samples[it, kk]
            m_range = np.arange(max_m) + 1
            log_s = np.array([self.stir_.stirlog(max_m, m) for m in m_range])
            m_dist = np.exp( logg(abk) - logg(abk + max_m) +
                             log_s + m_range * np.log(abk) )
            """MOSTLY FIXED.  m_dist should be a proper distribution"""
            m_dist[np.isnan(m_dist)] = 0
            m_dist += 1e-6
            
            mm1 = np.random.choice(m_range, p=m_dist/np.sum(m_dist))
            self.m_[jj, kk] = mm1

            #if verbose:
                #print(f"~~ restaraunt {jj}: {mm1} tables / {max_m} customers eating {kk}")
                #print(f"m_dist: {m_dist.round(3)}")
                
    
    def gibbs_cfr(self, x, j, iters, Tmax=None, Kmax=None, resume=False, verbose=False):
        """
        Runs the Gibbs sampler to generate posterior estimates of t and k.
        x: data matrix, stored row-wise if multidimensional
        j: vector of group labels; must have same #rows as x
        iters: number of iterations to run
        Tmax: maximum number of clusters for each group
        Kmax: maximum number of atoms to draw from base measure H
        resume: if True, will continue from end of previous direct_samples, if dimensions match up
        
        returns: this HDP object with cfr_samples attribute
        """
        
        group_counts = pd.Series(j).value_counts()
        J, N = np.max(j) + 1, len(j)
        # Set default Tmax and Kmax, if not provided
        if Tmax is None: Tmax = min(100, np.max(group_counts))
        if Kmax is None: Kmax = min(100, N)
            
        self.n_ = np.zeros((J, Tmax), dtype='int')
        self.m_ = np.zeros((J, Kmax), dtype='int')
        self.cfr_samples = np.zeros((iters+1, N, 3), dtype='int')
        self.cfr_samples[:,:,0] = j
        np.seterr('ignore')
        
        # Set random initial values for t and k assignments
        t0, k0 = self.cfr_samples[0,:,1], self.cfr_samples[0,:,2]
        t0[:] = np.random.randint(0, Tmax, size=N)
        self.tk_map_ = np.random.randint(0, Kmax//2, (J, Tmax))
        self.tally_up(it=0, which='n')
        for jj in range(J):
            for tt in np.where(self.n_[jj, :] > 0)[0]:
                #print(f"mapping: {(jj, tt)} -> {self.tk_map_[jj, tt]}")
                k0[np.logical_and(j == jj, t0 == tt)] = self.tk_map_[jj, tt]
        self.tally_up(it=0, which='m')
        
        for s in range(iters):
            t_prev, k_prev = self.cfr_samples[s,:,1], self.cfr_samples[s,:,2]
            t_next, k_next = self.cfr_samples[s+1,:,1], self.cfr_samples[s+1,:,2]
            # Copy over the previous iteration as a starting point
            t_next[:], k_next[:] = t_prev, k_prev
            
            self.draw_t(s+1, x, j, Tmax, Kmax, verbose)
            self.draw_k(s+1, x, j, Kmax, verbose)
        
        self.cfr_samples = self.cfr_samples[1:,:,1:]
        return self  
    
    
    def gibbs_direct(self, x, j, iters, Kmax=None, resume=False, verbose=False):
        """
        Runs the Gibbs sampler to generate posterior estimates of k.
        x: data matrix, stored row-wise if multidimensional
        j: vector of group labels; must have same #rows as x
        iters: number of iterations to run
        Kmax: maximum number of atoms to draw from base measure H
        resume: if True, will continue from end of previous direct_samples, if dimensions match up
        
        returns: this HDP object with direct_samples attribute
        """
        
        prev_direct, prev_beta = None, None
        start = 0
        if resume == True:
            # Make sure the x passed in is the same size as it previously was
            assert (x.shape[0] == self.direct_samples.shape[1] and
                    Kmax == self.beta_samples.shape[1] - 1), "Cannot resume with different data."
            iters += self.direct_samples.shape[0]
            prev_direct, prev_beta = self.direct_samples, self.beta_samples
            start = self.direct_samples.shape[0]
        
        group_counts = pd.Series(j).value_counts()
        J, N = np.max(j) + 1, len(j)
        if Kmax is None: Kmax = min(100, N)
            
        self.direct_samples = np.zeros((iters+1, N, 2), dtype='int')
        self.direct_samples[:,:,0] = j
        self.beta_samples = np.zeros((iters+1, Kmax+1))
        self.stir_ = StirlingEngine(np.max(group_counts) + 1)
        np.seterr('ignore')
        
        if resume == True:
            # Fill in the start of the samples with the previously computed samples
            self.direct_samples[1:start+1,:,1] = prev_direct
            self.beta_samples[1:start+1,:] = prev_beta
            # q_ and m_ attributes should already still exist within the object
        else:
            self.q_ = np.zeros((J, Kmax), dtype='int')   # performs the same function as n_
            self.m_ = np.zeros((J, Kmax), dtype='int')
            
            # Set random initial values for k assignments
            k0 = self.direct_samples[0,:,1]
            k0[:] = np.random.randint(0, Kmax, size=N)
            self.tally_up(it=0, which='q')
            # Implicitly set random t assignments by drawing possible m counts (m_jk <= q_jk)
            for jj in range(J):
                for kk in range(Kmax):
                    max_m = self.q_[jj, kk]
                    if max_m == 1:
                        self.m_[jj, kk] = 1
                    elif max_m > 1:
                        self.m_[jj, kk] = np.random.randint(1, max_m)
            # Compute the corresponding beta values from m assignments
            Mk = np.sum(self.m_, axis=0)
            self.beta_samples[0,:] = np.random.dirichlet(np.append(Mk, self.g_) + 1e-10)
        
        for s in range(start, iters):
            # Copy over the previous iteration as a starting point
            self.direct_samples[s+1,:,1] = self.direct_samples[s,:,1] 
            self.beta_samples[s+1,:] = self.beta_samples[s,:]
            
            self.draw_z(s+1, x, j, Kmax, verbose)
            self.draw_m(s+1, x, j, Kmax, verbose)
            
            Mk = np.sum(self.m_, axis=0)
            # Dirichlet weights must be > 0, so in case some k is unused, add epsilon
            self.beta_samples[s+1,:] = np.random.dirichlet(np.append(Mk, self.g_) + 1e-10)
            if verbose: print(self.beta_samples[s+1,:].round(3))
        
        self.direct_samples = self.direct_samples[1:,:,1]
        self.beta_samples = self.beta_samples[1:,:]
        return self

In [76]:
# Simulated data (Poisson example)
N = 200
np.random.seed(0)
j = np.random.randint(0, 9, N)
x = np.random.poisson(j, N)
data = np.c_[x, j]

%time c = HDP(f='poisson', hypers=(1,10)).gibbs_direct(x[:,None], j, iters=10, Kmax=10)
c.direct_samples.shape

Wall time: 2.65 s


(10, 200)

In [77]:
%time c.gibbs_direct(x[:,None], j, iters=10, resume=True, Kmax=10)
c.direct_samples.shape

Wall time: 2.58 s


(20, 200)

In [78]:
print(c.beta_samples.round(3))

[[0.098 0.127 0.254 0.035 0.076 0.039 0.159 0.026 0.066 0.119 0.001]
 [0.    0.    0.239 0.085 0.119 0.052 0.285 0.026 0.047 0.087 0.06 ]
 [0.    0.031 0.01  0.043 0.198 0.046 0.113 0.152 0.039 0.355 0.013]
 [0.    0.044 0.027 0.145 0.073 0.043 0.221 0.048 0.034 0.215 0.15 ]
 [0.    0.179 0.019 0.181 0.039 0.    0.254 0.005 0.04  0.279 0.006]
 [0.    0.151 0.062 0.014 0.    0.    0.429 0.    0.053 0.2   0.092]
 [0.    0.04  0.122 0.01  0.    0.    0.473 0.    0.    0.354 0.002]
 [0.    0.253 0.015 0.041 0.    0.    0.415 0.    0.    0.135 0.141]
 [0.    0.012 0.071 0.069 0.    0.    0.333 0.    0.    0.331 0.185]
 [0.    0.    0.258 0.036 0.    0.    0.392 0.    0.    0.226 0.088]
 [0.    0.    0.046 0.051 0.    0.    0.131 0.    0.    0.665 0.106]
 [0.    0.044 0.117 0.162 0.    0.    0.387 0.    0.    0.254 0.036]
 [0.    0.029 0.148 0.02  0.    0.    0.296 0.    0.    0.399 0.108]
 [0.    0.023 0.017 0.046 0.    0.    0.564 0.117 0.005 0.175 0.051]
 [0.    0.073 0.062 0.165 0.    0.

In [140]:
# Simulated data (Multinomial example)
N, L = 200, 100
np.random.seed(1)
X = np.random.randint(0, 10, (N, L))
j = np.random.randint(0, 10, N)

c = HDP(gamma=5, f='multinomial', hypers=(L, np.full(L, 1/L)))
%time c = c.gibbs_direct(X, j, iters=10)

Wall time: 6.15 s


## Latent Topic Modeling Application

Below is an application of the above sampler using a multinomial data model.  The data is `final_project_data.csv`, produced by the modified preprocessing code in this directory, which contains a `(J, L)` matrix in which entry `(j,l)` contains the count of word `l` in document `j`, with the corresponding words given in the column names.

For the Dirichlet prior here, we use the observed distribution of the corpus vocabulary over all documents.  Customers could be encoded in four different ways to compare performance:
+ As a single word (such that `f` is categorical)
+ As a set of all identical words within a given document (each row of the data matrix has one entry, but the value can vary)
+ As a set of all words in a single sentence
+ As the entire document (essentially making this a non-hierarchical DP)

Since this algorithm has not been optimized, only a subset of the full dataframe is used for now.

In [60]:
full_df = pd.read_csv('final_project_data.csv', index_col=0, dtype='int')
vocab = full_df.columns

In [61]:
def expand_doc(doc_in):
    """Expands a row (passed in a series) into a dataframe in which each row contains one word."""
    
    doc_in = doc_in.drop(J_ID, axis=1)
    doc_in = doc_in.iloc[0, :]
    
    # Convert from one row per unique word to one row per individual word
    each_word = []
    while len(doc_in) > 0:
        doc_in = doc_in[doc_in > 0]
        each_word += list(doc_in.index)
        doc_in -= 1
        
    doc_out = pd.DataFrame(np.zeros((len(each_word), len(vocab)), dtype='int'),
                           columns=vocab, dtype='int')
    for i, word in enumerate(each_word):
        doc_out.loc[i, word] = 1
    return doc_out

J_ID = 'document#'
if J_ID not in full_df.columns:
    full_df.insert(0, column=J_ID, value=full_df.index)

Jmax = 5
wordset_df = full_df.iloc[:Jmax,:].groupby(J_ID).apply(expand_doc)

In [141]:
ji_indices = wordset_df.index.to_frame()
j = np.array(ji_indices[J_ID])
X = np.array(wordset_df)
# Get the corresponding word each ji is associated with
ji_words = vocab[np.where(X > 0)[1]]

Tmax, Kmax = np.max(ji_indices[1]), 20
# Get a prior distribution over the vocabulary from selected documents
L, h_alpha = X.shape[1], np.sum(X, axis=0)

In [142]:
%time hdp = HDP(f='categorical', hypers=(L, h_alpha)).gibbs_direct(X, j, 10, Kmax=10)

Wall time: 4.26 s


In [123]:
# Given seating assignment in final iteration, cluster customers with same k value
k_final = hdp.direct_samples[-1,:,1]
clusters = pd.DataFrame({'doc': j, 'word': ji_words, 'cluster': k_final})
for k in set(k_final):
    clusters_k = clusters[clusters['cluster'] == k].sort_values('word')
    print(f"-----------\nk = {k} (size {clusters_k.shape[0]})")
    print(clusters_k)

### Speed Testing

In [127]:
from scipy.sparse import csr_matrix

# Unit efficiency tests for direct sampling
%time hdp = HDP(f='multinomial', hypers=(L, h_alpha)).gibbs_direct(X, j, 100)
%time hdp = HDP(f='multinomial', hypers=(L, h_alpha)).gibbs_direct(csr_matrix(X), j, 100)
%time hdp = HDP(f='categorical', hypers=(L, h_alpha)).gibbs_direct(X, j, 100)
%time hdp = HDP(f='categorical', hypers=(L, h_alpha)).gibbs_direct(csr_matrix(X), j, 100)

Wall time: 11.5 s
Wall time: 20 s
Wall time: 3.86 s
Wall time: 10.4 s


In [128]:
# Unit efficiency tests for CFR sampling vs direct sampling
%time hdp = HDP(f='multinomial', hypers=(L, h_alpha)).gibbs_cfr(X, j, 100)
%time hdp = HDP(f='multinomial', hypers=(L, h_alpha)).gibbs_direct(X, j, 100)
%time hdp = HDP(f='categorical', hypers=(L, h_alpha)).gibbs_cfr(X, j, 100)
%time hdp = HDP(f='categorical', hypers=(L, h_alpha)).gibbs_direct(X, j, 100)

Wall time: 16.5 s
Wall time: 11.5 s
Wall time: 8.49 s
Wall time: 3.85 s


In [93]:
np.random.seed(0)

N, L, Jmax, Kmax = 100000, 5000, 50, 100
j = np.random.choice(Jmax, size=N)
k = np.random.choice(Kmax, size=N)
ha = np.ones(L)

Xtest = np.zeros((N, L), dtype='int')
col_choices = np.random.choice(L, size=N)
Xtest[range(N), col_choices] = 1
Xtest_sparse = csr_matrix(X)

# Unit efficiency tests for mixture components
%time fk = mnom_fk_cust(0, Xtest, k, Kmax, L, ha)
%time fk = mnom_fk_cust(0, csr_matrix(Xtest), k, Kmax, L, ha)
%time fk = cat_fk_cust(0, Xtest, k, Kmax, L, ha)
%time fk = cat_fk_cust(0, csr_matrix(Xtest), k, Kmax, L, ha)

Wall time: 1.91 s
Wall time: 2.6 s
Wall time: 34.9 ms
Wall time: 2.57 s
